In [38]:
import torch, time, pickle,os
import torch_geometric as tg
from torch_geometric.data import Data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import os.path as osp
import networkx as nx
from sklearn.metrics import r2_score
path=osp.expanduser('~/../../tigress/cj1223/SAM_runs')

log = np.array([2, 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 29])

targets = np.array([8,15,20,21,23,27])

In [2]:
runs=os.listdir(path)
runs

['run_3',
 'run_2',
 'run_5',
 'run_4',
 'run_8',
 'run_10',
 'run_7',
 'fiducial',
 'run_6',
 'run_1',
 'run_9']

In [4]:
dfs = []
# runs=['run_2', 'run_6', 'run_7']
for run in tqdm(runs, total=len(runs)):
# run=runs[0]
    dat = []
    for i in tqdm(range(5)):
        for j in range(5):
            for k in range(5):
                ex = f'{i}_{j}_{k}/galprop_99-99.dat'
                tpath = osp.join(path,run)
                tpath = osp.join(tpath,ex)
                pdc=pd.read_table(tpath, skiprows=0, delimiter=',', nrows=41, header=None)
                newcols=pdc.iloc[:,0]
                pds=pd.read_table(tpath, skiprows=41, delimiter='\s+', header=None)
                pds.columns=np.array(newcols)
                mask = np.logical_and(pds[newcols[5]]>10**(10-9), pds[newcols[5]]<10**(13.8-9) ,pds[newcols[4]]==0)
                pdhalo=pds[mask]
                dat.append(pdhalo)
    df = pd.concat(dat)
    df = np.log10(df[newcols[log]])
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.dropna(axis=0)
#     dfdf[newcols[targets]]
    dfs.append(df)

100%|█████████████████████████████████████████████████████████████████| 5/5 [00:35<00:00,  7.13s/it]
/home/cj1223/.conda/envs/juptorch/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:29<00:00,  5.96s/it]
/home/cj1223/.conda/envs/juptorch/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:29<00:00,  5.80s/it]
/home/cj1223/.conda/envs/juptorch/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:31<00:00,  6.24s/it]
/home/cj1223/.conda/envs/j

100%|█████████████████████████████████████████████████████████████████| 5/5 [00:35<00:00,  7.14s/it]
/home/cj1223/.conda/envs/juptorch/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:32<00:00,  6.42s/it]
/home/cj1223/.conda/envs/juptorch/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)
100%|███████████████████████████████████████████████████████████████| 11/11 [06:03<00:00, 33.06s/it]


In [5]:
ids = np.hstack([10**d[newcols[2]] for d in dfs])
vals, counts = np.unique(ids, return_counts = True)

In [6]:
mask=counts==11
len(vals[mask]), sum(mask)

(156313, 156313)

In [7]:
all_vals=vals[mask]
DFs = []
for df in dfs:
    mask = np.isin(10**df[newcols[2]], all_vals)
    d = df[mask].reset_index()
    d[newcols[20]]-=d[newcols[15]]
    DFs.append(d)

In [8]:
for df in DFs:
    print(len(df))

156313
156313
156313
156313
156313
156313
156313
156313
156313
156313
156313


In [9]:
DFs[1][newcols[targets]]-DFs[0][newcols[targets]]

,# 8 mstar stellar mass [1.0E09 Msun],# 15 mcold cold gas mass in disk [1.0E09 Msun],# 20 Metal_cold metal mass in cold gas [Zsun*Msun],# 21 sfr instantaneous SFR [Msun/yr],# 23 sfrave100myr SFR averaged over 100 Myr [Msun/yr],# 27 mBH black hole mass [1.0E09 Msun]
0,-0.003588,0.002421,-0.005259,-0.001465,-0.002548,0.034686
1,0.001240,-0.003021,-0.000439,-0.006457,-0.006681,0.000039
2,-0.004178,0.001877,0.010881,-0.009047,-0.009593,-0.066785
3,0.001012,0.011145,-0.009012,0.017582,0.017713,0.342194
4,0.003068,-0.002029,0.003001,-0.000557,-0.000547,-0.066939
...,...,...,...,...,...,...
156308,0.000130,-0.000052,0.000021,-0.000073,-0.000070,0.000056
156309,0.000197,-0.000065,0.000659,0.000833,0.000837,-0.000035
156310,0.000955,0.005399,0.000014,0.012620,0.012683,-0.000013
156311,0.001692,-0.001586,0.001556,-0.000621,-0.000607,0.041387


In [52]:
stds = []
mus = []
rhos = []
r2s = []
for i in range(len(DFs)):
    for j in range(i+1, len(DFs)):
        y = DFs[i][newcols[targets]]
        pred = DFs[j][newcols[targets]]
        d = y-pred
        a=d.describe()
        stds.append(np.array(a.iloc[2]))
        mus.append(np.abs(np.array(a.iloc[1])))
        r2 = []
        rho = []
        for k in range(len(targets)):
            r2.append(r2_score(DFs[i][newcols[targets[k]]],DFs[j][newcols[targets[k]]]))
            rho.append(np.corrcoef(DFs[i][newcols[targets[k]]],DFs[j][newcols[targets[k]]])[0][1])
        r2s.append(np.array(r2))
        rhos.append(np.array(rho))    
        

In [56]:
std=np.vstack(stds)
mean=np.vstack(mus)
rho=np.vstack(rhos)
r2=np.vstack(r2s)

In [57]:
mu, mi, ma = np.mean(std, axis=0), np.min(std, axis=0), np.max(std, axis=0)
mu

array([0.04306521, 0.09554877, 0.07473999, 0.23026338, 0.19981449,
       0.12952461])

In [58]:
mu, mi, ma = np.mean(mean, axis=0), np.min(mean, axis=0), np.max(mean, axis=0)
mu

array([9.70155668e-05, 2.18131216e-04, 1.08137429e-04, 1.87702573e-04,
       2.60781194e-04, 3.36672551e-04])

In [59]:
mu, mi, ma = np.mean(rho, axis=0), np.min(rho, axis=0), np.max(rho, axis=0)
mu

array([0.9989633 , 0.98360206, 0.99024077, 0.97356969, 0.97990595,
       0.96658409])

In [60]:
mu, mi, ma = np.mean(r2, axis=0), np.min(r2, axis=0), np.max(r2, axis=0)
mu

array([0.99792635, 0.96720197, 0.98048079, 0.94711836, 0.95979897,
       0.93318725])

In [36]:
# results = {}
# results['ys'] = np.array(DFs[0][newcols[targets]])
# results['pred'] = np.array(DFs[1][newcols[targets]])

# folder = 'results_SAM'
# run = 'SAM01'
# pointer=osp.expanduser(f'~/../../scratch/gpfs/cj1223/GraphResults/{folder}/{run}')   
# with open(osp.join(pointer, 'result_dict.pkl'), 'wb') as handle:
#     pickle.dump(results, handle)